# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **

* The problem of identifying the students who might need early intervention before they fail to graduate given sample data is a ** supervised classification problem**. The identification of students who may need early intervention is the possibility of a student weather the student will pass or fail given their data. Since we have only two possibilities ie) the student will pass or fail, this problem has two possible outcomes 'yes' or 'no' for the prediction results. The process of predicting discrete values in this case 'yes' or 'no' is a classification problem. Here we map the input features to a set of discrete output values. *

* We trying to predict discrete binary outputs, Weather the will student pass (and would not need an intervention), or the student will not pass (and would require an intervention). *


## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [58]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score
    

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [59]:
# TODO: Calculate number of students
n_students = len(student_data)

# TODO: Calculate number of features
n_features = len(student_data.columns) - 1 #subtract traget column

# TODO: Calculate passing students
n_passed = len(student_data[student_data["passed"] == 'yes'])

# TODO: Calculate failing students
n_failed = len(student_data[student_data["passed"] == 'no'])

# TODO: Calculate graduation rate
grad_rate = ( float(n_passed) / float(n_students) ) * 100 #percentage of no of student who passed

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [60]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [61]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [62]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import StratifiedShuffleSplit

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above

shuffle_split = StratifiedShuffleSplit(y_all,test_size=95,random_state=42)

for train_index, test_index in shuffle_split:
    X_train, X_test = X_all.iloc[train_index], X_all.iloc[test_index]
    y_train, y_test = y_all[train_index], y_all[test_index]

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


# Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: ** 

* The learning models i think which are appropriate for this binary classification problem are **Decision Tree learning model**, **Support Vector Machines** learning model and **Ensemble Bagging classification learning model** with K-Nearest Neighbor estimator. *

* **Decision Trees Classification learning models** are simple yet most successful form of non-parametric learning algorithm. A Decision Tree learning model maps the input situation described by a set of attributes to an output called decision which is the predicted output value for the input. A decision tree learning model reaches its decision by performing a sequence of test. Each node in the decision tree corresponds to a test of the value of one the input properties, and the branches from the node are assigned with the possible values of the test. If a node doesn't contain any node then it is called a leaf node and it holds the value to be returned of any particular test. Decision Tree learning models create a decision boundary surface form the sample data and predicts the class of the unseen input data based on the boundary surface developed from the sample data. Advantages of Decision Tree learning model is that they are simple to understand and can easily visualised. These models will be able to handle both the numerical and categorical data. Disadvantages of these learning models are that they can create over complex tress that do not generalize the data well and suffer from overfitting. Since the decisions are made at each node locally, it cannot guarantee to produce globally optimal decision trees. Especially, these models can create biased decision trees if some classes in the data dominate. ie, Decision trees are not good for data with skewed distributions. Since the cost of using and predicting the data is logarithmic in the number of data points used to train the tree, it is cost efficient and considering the school has limited computational resources this model makes a good fit for this problem. Also our data has more number of categorical data and we have to predict the discrete output weather the student will pass or fail, this make this learning model a good candidate for this given problem. Some of the real world applications of this learning model are in the domain of Business management in paricular with customer relationship management and fraud detection. Reference: http://what-when-how.com/artificial-intelligence/decision-tree-applications-for-data-modelling-artificial-intelligence/. The training time for this model is O(n_{features}n_{samples}^{2}\log(n_{samples})). The query time is O(\log(n_{samples})).*

* **Support Vector Machines** are supervised learning models can be used for both the classification and regression tasks. However, in this section we will focus only on using this learning model for classification problem since our goal is to predict discrete ouput values. ie, the student will pass or not. This learning algorithm tries to build a model that represents the data given beloning to each of the classes. This model represents the examples as data points in space, mapped such that each seperate categories are made distinct by clear gaps between each of the classes which as wide as possible. New unseen example data are mapped into the same space and predicted to which category they belong. This learning model is particularly useful to classify linear data points. However, this model can also perform well with non-linear classification using a trick called kernal trick in which the model makes the data points to be linearly seperable in a higher dimensional space according to the input feature vector. Some of the real world implementation of support vector machine learning models are in the area of text categorization, image recognition, bio informatics. However this type of models are not limited to these applications and can be used in many other ways. Text categorization is the task of classifying text documents into a predefiend number of given categories. Reference - https://arxiv.org/pdf/math/0612817.pdf . Advantages of SVM's are that these models are effective with high dimensional spaces even is the number of features is greater than the number of samples. When prediction a new unseen data, it only uses a subset of the sample space called support vectors, these model are memory efficient.These models are highly versatile in terms of defining the kernal functions which makes it to adjust to broad array of applications. Disadvantages are that these models give poor performance if the number of features are very much greater than the sample space. Also, these models do not provide direct probability estimates, but these can be calculated using expensive k-fold cross validation techniques.Since our data of students has a good number of features, this model promotes itself as a good candidate of the given problem. This model has a space complexity of O(n^2) where n is the input data size. It has a training time of O(n^3) where n is the training dataset size.*

* **Ensemble Bagging classifier** with K-Nearest Neighbors estimator. The goal of ensemble methods is to combine the predictions of several base estimators built with a given learning alogrithm (in our case KNN) in order to improve the generalizability or to make more robust when compared to a single base estimator. There are two types of ensemble methods. One is averaging method where several estimators are build and their predictions are averaged. Other method of ensemle learning is boosting method where multiple base estimators built sequentialy one over the other and tries to reduce the bias of the previously combined estimator. In this problem we will proceed with ensemble bagging classifier combined with K-Nearest Neighbors classification learning model. The K-nearest neighbor classification model is a type of instance based learning where the generalized model will not be constructed, instead the model will store all the training instances and classification of new unseen data are computed from simple majority votes of the nearest neighbors of each data point. Here k, means the number of nearest neighbors to be considered. The advantages of using ensemble bagging classification is that multiple base estimator's predictions are averaged and usually this increases the generalizability of the learning model. Disadvantages are that in order to average the predictions of multiple base estimators, the same has to be be calculated so the running time of this model is usually high. Real world implementations of these kinds of learning models are some in the areas of Remote Sensing, Person recognition, One Vs all recognition, Medicine which require large number of inputs, large number of features. Finger print recognition, face recognition are some applications of ensemble methods in the domain of person recognition. Reference - http://www.ehu.eus/ccwintco/uploads/1/1f/Eider_ClassifierEnsembles_SelectRealWorldApplications.pdf .*

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [66]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.6f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "\nMade predictions in {:.6f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .\n".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.6f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.6f}.\n".format(predict_labels(clf, X_test, y_test))


### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [67]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier

## I Tried Many models for practicing
#from sklearn.naive_bayes import GaussianNB
#from sklearn.ensemble import AdaBoostClassifier
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.ensemble import GradientBoostingClassifier
#from sklearn.linear_model import SGDClassifier

# TODO: Initialize the three models
svm_clf = SVC()
des_tree = DecisionTreeClassifier(max_depth=8,min_samples_split=10)
ensemble_bagging = BaggingClassifier(KNeighborsClassifier(n_neighbors=3),max_samples=0.5, max_features=2)

## I Tried Many models for practicing
#gaussian_bayes = GaussianNB()
#knn = KNeighborsClassifier(n_neighbors=5)
#ensemble_adaboost = AdaBoostClassifier(n_estimators=100)
#ensemble_random_forest = RandomForestClassifier(n_estimators=10, max_depth=8,min_samples_split=10, random_state=0)
#ensemble_gradient_descent = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=4, random_state=0)
#sgdc = SGDClassifier(loss="hinge", penalty="l2")


# TODO: Set up the training set sizes
training_sizes = [100, 200, 300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)
clf_models = { "DecisionTree Classification": des_tree, \
               "Svm Classification": svm_clf,    \
               "Ensemble Bagging Classification": ensemble_bagging,    \
                #"Ensemble AdaBoost Classification": ensemble_adaboost,    \
                #"Ensemble Random Forest Classification": ensemble_random_forest,    \
                #"Ensemble Gradient Descent Classification": ensemble_gradient_descent,    \
                #"Stochastic Gradient Descent Classification": sgdc,    \
                #"Gaussian Naive Bayes Classification": gaussian_bayes,    \
                #"KNN Classification": knn,    \
             }

for name, clf in clf_models.items():
    print "\n============================================================\n"
    print "{} Model \n".format(name)
    for batch in training_sizes:
        print "********************************************\n"
        train_predict(clf, X_train[:batch], y_train[:batch], X_test, y_test)



Svm Classification Model 

********************************************

Training a SVC using a training set size of 100. . .

Trained model in 0.002000 seconds

Made predictions in 0.001000 seconds.
F1 score for training set: 0.881119.

Made predictions in 0.001000 seconds.
F1 score for test set: 0.815789.

********************************************

Training a SVC using a training set size of 200. . .

Trained model in 0.004000 seconds

Made predictions in 0.003000 seconds.
F1 score for training set: 0.875000.

Made predictions in 0.003000 seconds.
F1 score for test set: 0.823529.

********************************************

Training a SVC using a training set size of 300. . .

Trained model in 0.010000 seconds

Made predictions in 0.007000 seconds.
F1 score for training set: 0.878587.

Made predictions in 0.003000 seconds.
F1 score for test set: 0.791946.



DecisionTree Classification Model 

********************************************

Training a DecisionTreeClassifier usin

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Support Vector Machines**  

| Training Set Size | Training Time           | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |           0.0010        |        0.0020          |       0.8811     |       0.8158    |
| 200               |           0.0040        |        0.0020          |       0.8750     |       0.8235    |
| 300               |           0.0110        |        0.0020          |       0.8786     |       0.7919    |

** Classifer 2 - Decision Tree Classifier **

| Training Set Size | Training Time           | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |           0.0010        |        0.0000          |       0.9612     |      0.6206     |
| 200               |           0.0010        |        0.0000          |       0.9353     |      0.6774     |
| 300               |           0.0020        |        0.0000          |       0.8840     |      0.7538     |

** Classifer 3 - Ensemble Bagging Classifier **

| Training Set Size | Training Time           | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |           0.0170        |        0.0040          |       0.7600     |      0.7346     |
| 200               |           0.0090        |        0.0160          |       0.8039     |      0.7916     |
| 300               |           0.0150        |        0.0160          |       0.8040     |      0.8050     |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: ** 

* I choose the **Support Vector Machine** classifier as the best classification model for the given problem of classifying weather a student will pass or not based on the experiments I performed with various learning models.*

* Since the **Decision Tree** doesn't give good accuracy because our given data is skewed. ie) more number of students have passed the examinations *

* **Ensemble Bagging Classifier** is giving more or less the same accuracy when compared to the support vector machines but the running time for model training and predicting the data is relatively high. Considering the fact that the school has limited computing resources, We cannot recomnd Ensemble Bagging Classifer. *

* On the other hand ** Support Vector Machines ** is giving out good accuracy based of the f1 score and also has smaller training time and a constant prediction time independent of the input size, this model is performing very well in terms of cost effectiveness, time and space complexity. Also we can further fine tune the model for better accuracy. Also we have very good number of features for predicting the output. So, SVM is the optimal and best model to proceed with.*


### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

* Support vector machines is a machine learning algorithm which can be used for classifying a certain input to which category it belongs to. It works by building a model which tries to create a boundary surface between the categories that maximizes the distance between the categories in the given sample data. The model is created by mapping the input features to the ouptut categories using mathematical relations. Then this learning model using the boundary surface it created, can be used to classify new unseen data to which category it belongs to by predicting where the new input map to the output categories which it has created during the model building phase called as training phase. *

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [77]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score

# TODO: Create the parameters list you wish to tune
parameters = {'kernel':('linear','rbf', 'poly','sigmoid'), 'C':[1, 50], 'degree':[3,6],'gamma':[0.0001,0.00010]}

# TODO: Initialize the classifier
clf = SVC()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf,parameters,scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_

#print "--- ",grid_obj.score(X_test, y_test)


# Report the final F1 score for training and testing after parameter tuning
print "Tuned model parameters are {}".format(grid_obj.best_params_)
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Tuned model parameters are {'kernel': 'rbf', 'C': 50, 'gamma': 0.0001, 'degree': 3}

Made predictions in 0.007000 seconds.
Tuned model has a training F1 score of 0.8174.

Made predictions in 0.002000 seconds.
Tuned model has a testing F1 score of 0.8182.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

* The final model's F1 score for training data is 0.8786 and the F1 score for the testing data is 0.8182 . The final model's best parameters are Kernel is rbf, C is 1, degrees of polynomial is 3 and gamma ( Kernel Coeff ) is 0.0001 . The final tuned model's performance using F1 score is 0.8182 which approximately 1% more than than of untuned model*


> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.